<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_MultiTimeFrame_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mplfinance

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
#from google.colab import drive
#drive.mount('/content/drive')

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/'

mypf = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

#mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]
#mypf = mypf[~mypf['Criteria'].isin(['V40','V40N'])]
mypf = mypf[mypf['TFrame'] == 'W']

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC','JIOFIN','SCHAEFFLER ','ADANITRANS', 'EQUITAS']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

all_prospects = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

all_prospects.sort()
#stock_ids

In [3]:
import pandas as pd
import mplfinance as mpf
import yfinance as yfin
#import talib
yfin.pdr_override()

def get_mtm_data(symbol, tf):
  if tf == 'D':
    intvl, sma, lma, xlma, prd, start_date, tfc = '1d', 20, 50, 200, '2y', '2023-01-01','Daily'
  elif tf == 'W':
    intvl, sma, lma, xlma, prd, start_date, tfc = '1wk', 5, 10, 30, '6y', '2019-01-01','Weekly'
  else:
    intvl, sma, lma, xlma, prd, start_date, tfc = '1mo', 2, 5, 10, '10y', '2013-01-01','Monthly'

  # Sample data with OHLCV (Open, High, Low, Close, Volume) prices
  # Sample data
  df = yfin.Ticker(symbol+'.NS').history(period=prd, interval=intvl)[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
  #df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
  #df = pd.DataFrame(data)
  #df.set_index('Date', inplace=True)

  # Calculate moving averages
  df['MA_short'] = df['Close'].rolling(window=sma).mean()
  df['MA_long'] = df['Close'].rolling(window=lma).mean()
  df['MA_xlong'] = df['Close'].rolling(window=xlma).mean()

  df = df[df.index >= start_date]
  col = tf + '_' + 'Flag'
  df[col] = 'R'
  df.loc[df['MA_short'] > df['MA_long'], col] = 'G'
  df = df[df.index > '31-Jan-2023']
  cols = [col]
  return df[df.index > '31-Jan-2023'][cols]

In [4]:

def get_stock_buy_sell(symbol):
  m_df = get_mtm_data(symbol, 'M')
  w_df = get_mtm_data(symbol, 'W')
  d_df = get_mtm_data(symbol, 'D')
  m_df['month'] = m_df.index.month
  w_df['week'] = w_df.index.week
  d_df['week'] = d_df.index.week
  d_df['month'] = d_df.index.month
  d_df = d_df.reset_index()
  cmb_df = pd.merge(d_df, w_df, on='week', how='outer')
  cmb_df = pd.merge(cmb_df, m_df, on='month', how='outer')
  cols = ['Date','D_Flag','W_Flag','M_Flag']
  cmb_df = cmb_df[cols]
  cmb_df.set_index('Date', inplace=True)
  df = yfin.Ticker(symbol+'.NS').history(period='1y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
  df = df.join(cmb_df)
  df['F_Flag'] = 'R'
  df['G_Close'] = df['Close'].min()
  df.loc[(df['D_Flag'] == 'G') & (df['W_Flag'] == 'G') & (df['M_Flag'] == 'G'), 'F_Flag'] = 'G'
  df.loc[(df['F_Flag'] == 'G'), 'G_Close'] = df['Close'].max()
  df['Prev_F_Flag'] = df['F_Flag'].shift(1)
  df_reco = df[df['F_Flag'] != df['Prev_F_Flag']]
  df_reco['Signal'] = df_reco['F_Flag'].apply(lambda x: 'Buy' if x == 'G' else 'Sell')
  df_reco.drop(['Open','Low','High','Volume'], axis=1, inplace=True)
  return df_reco.tail(1)

def get_weekly_buy_sell():
    df_buy_sell = pd.DataFrame()
    for stock_id in all_prospects:
        df_tmp = get_stock_buy_sell(stock_id)
        df_tmp['Stock'] = stock_id
        df_buy_sell = df_buy_sell.append(df_tmp)
    df_buy_sell.drop(['D_Flag','W_Flag','M_Flag','G_Close','Prev_F_Flag'], axis=1, inplace=True)
    return df_buy_sell.sort_index()

df_weekly_buy_sell = get_weekly_buy_sell()
df_weekly_buy_sell

,Close,F_Flag,Signal,Stock
Date,,,,
2023-02-27 00:00:00+05:30,657.952759,R,Sell,RAJESHEXPO
2023-05-02 00:00:00+05:30,1138.411133,G,Buy,INDUSINDBK
2023-05-02 00:00:00+05:30,2907.300537,G,Buy,DIXON
2023-05-02 00:00:00+05:30,90.750000,G,Buy,PARAGMILK
2023-05-02 00:00:00+05:30,146.431808,G,Buy,GICRE
...,...,...,...,...
2024-01-01 00:00:00+05:30,799.250000,G,Buy,ROSSARI
2024-01-01 00:00:00+05:30,4962.450195,G,Buy,FINEORG
2024-01-01 00:00:00+05:30,1493.199951,R,Sell,INDIGOPNTS


In [5]:
df_weekly_buy_sell[df_weekly_buy_sell['Signal'] == 'Sell'].tail(20)

,Close,F_Flag,Signal,Stock
Date,,,,
2023-09-11 00:00:00+05:30,921.450012,R,Sell,RELAXO
2023-09-18 00:00:00+05:30,630.450012,R,Sell,STARHEALTH
2023-10-03 00:00:00+05:30,968.700012,R,Sell,WESTLIFE
2023-10-03 00:00:00+05:30,1878.300049,R,Sell,VINATIORGA
2023-10-03 00:00:00+05:30,8532.099609,R,Sell,CERA
2023-10-09 00:00:00+05:30,673.349976,R,Sell,VIPIND
2023-10-16 00:00:00+05:30,1796.949951,R,Sell,TATACOMM
2023-10-16 00:00:00+05:30,1255.050049,R,Sell,JCHAC
2023-10-18 00:00:00+05:30,636.224304,R,Sell,ARE&M


In [6]:
df_weekly_buy_sell[df_weekly_buy_sell['Signal'] == 'Buy'].tail(20)

,Close,F_Flag,Signal,Stock
Date,,,,
2023-12-12 00:00:00+05:30,88.250000,G,Buy,JMFINANCIL
2023-12-13 00:00:00+05:30,221.050003,G,Buy,BANKBARODA
2023-12-14 00:00:00+05:30,474.950012,G,Buy,VALIANTORG
2023-12-14 00:00:00+05:30,291.899994,G,Buy,CROMPTON
2023-12-14 00:00:00+05:30,148.250000,G,Buy,CASTROLIND
2023-12-18 00:00:00+05:30,253.500000,G,Buy,BANDHANBNK
2023-12-18 00:00:00+05:30,806.549988,G,Buy,SPORTKING
2023-12-18 00:00:00+05:30,220.500000,G,Buy,APEX
2023-12-22 00:00:00+05:30,1088.449951,G,Buy,TANLA
